**Log in to CATMAID instance**
<br>
After importing pymaid and contools, you must log in to your CATMAID instance. You will need to provide your username, password, token, and the url of the CATMAID instance.
Here, these data are stored in pymaid_creds.py and imported into the script. Make sure to add this filename to a .gitignore file to prevent sharing your credentials on github.

In [ ]:
import pymaid
import contools

from pymaid_creds import url, name, password, token
rm = pymaid.CatmaidInstance(url, token, name, password)

**Splitting Axons and Dendrites to generate axo-dendritic, axo-axonic, dendro-dendritic, and dendro-axonic adjacency matrices**
<br>
This requires a few different annotations and tags. First, you need the full list of neurons that you wish to split. In the below example, we combine two annotations using pymaid.get_skids_by_annotation(). You must then indicate your split tag (here, "mw axon split"), which is a previously generated node that indicates the putative axon initial zone. Everything upstream of this node will be considered dendrite and everything downstream axon.\
<br>
Next, you can indicate neurons with complex splits that use "axon start" and "axon end" tags to indicate where individual axon(s) start and end. Everything that is not an axon is considered a dendrite. Finally, indicate skeleton ids for neurons that have no split tags and shouldn't be split (not_split_skids).\
<br>
By running generate_adjs.adj_split_axons_dendrites(), you will generate and save CSVs for a-d, a-a, d-d, and d-a adjacency matrices in 'data/adj/...'

In [ ]:

all_neurons = pymaid.get_skids_by_annotation(['mw brain paper clustered neurons', 'mw brain accessory neurons'])
split_tag = 'mw axon split'
special_split_tags = ['mw axon start', 'mw axon end']
not_split_skids = pymaid.get_skids_by_annotation(['mw unsplittable', 'mw partially differentiated', 'mw brain incomplete'])

generate_adjs.adj_split_axons_dendrites(all_neurons, split_tag, special_split_tags, not_split_skids)

**Generating Filtered Edge List**
<br>
This code chunk will generate a edge list that is filtered based on a pair-wise threshold. This can be a synaptic threshold or fraction input threshold (meaning $\frac{synapses_{num}}{synapses_{total}-on-dendrite}$ or $\frac{synapses_{num}}{synapses_{total}-on-axon}$). You can choose between these using fraction_input=False (synaptic threshold) or fraction_input=True (fraction input threshold). The threshold variable will therefore be either synaptic strength or fraction input, respectively.

#insert diagram of pairwise threshold

Left-right homologous pairs must be provided as a CSV with columns "leftid" and "rightid". You must also provide annotations for all left or right hemisphere neurons. 

In [ ]:
# generate edge list with raw synapse threshold = 3
pairs_path = 'data/pairs/pairs-2021-04-06.csv'
pairs = contools.Promat.get_pairs(pairs_path=pairs_path)
generate_adjs.edge_thresholds(path='data/adj', threshold=3, left_annot='mw left', right_annot='mw right', pairs = pairs, fraction_input=False, date='2022-02-03')


# generate edge list with %input threshold = 0.01
pairs_path = 'data/pairs/pairs-2021-04-06.csv'
pairs = contools.Promat.get_pairs(pairs_path=pairs_path)
generate_adjs.edge_thresholds(path='data/adj', threshold=0.01, left_annot='mw left', right_annot='mw right', pairs = pairs, fraction_input=True, date='2022-02-03')